# Dark matter radius computation function
## Notebook content:
    1. imports
    2. get_sp()
    3. find functions for DM
    4. find functions for MASS

In [1]:
import yt
from yt .mods import *
import pandas as pd
import numpy

In [9]:
path = r'/vol/sci/astro/bigdata/tussbaum/halofinder/catalog/'
cen_gal_cat    = pd.read_pickle(path + 'cen_gal_cat.pkl')
sim_cat        = pd.read_pickle(path + 'sim_table.pkl')
sim_cat.set_index(['sid'], inplace=True)
sgal_cat       = pd.read_pickle(path + 'sat_gal_table.pkl')
sgal_cat.set_index(['sgal_id'], inplace=True)
uniq_cat       = pd.read_pickle(path + 'tgal_tmp_thick_table.pkl')
sgal_attributes_cat = pd.read_pickle(path + 'gal_R_attribute_cat.pkl')
sgal_attributes_cat.set_index(['sgal_id'], inplace=True)


/vol/sci/astro/home/adambeilialpha/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
gen="VELA_v2"
gal_num="15"
snapshot=0.24
galaxytype = 'Thick'

cen_gal_cat_s = cen_gal_cat[cen_gal_cat['type']==galaxytype][cen_gal_cat['gen']==gen][cen_gal_cat['galaxynumber']==gal_num][cen_gal_cat['aexp']==snapshot]
cen_i = cen_gal_cat_s.index[0]
ds = yt.load(sim_cat.loc[cen_i]["sim_path"].replace("from_nas2", "Users"))
center=cen_gal_cat_s[cen_col].values[0]#taking info written in important_col2 into a list
rad=cen_gal_cat_s['r_vir[kpc]'].values[0]
st.center= ds.arr(center, 'code_length').in_units('kpc')#giving center_new units of kpc (from code length)
st.sp=ds.sphere(center=center,radius=rad)

## The get_sp() function
The radius computation functions require a yt sphere object to acsses the right data for it to compute. The get_sp() func uses the imported catalouge for it to create such object. make sure you have the correct catalouge loaded into the variable cen_gal_cat_s.

In [10]:
def get_sp(cen_gal_cat_s):
#Get the angular momentum vector of the galaxy, and the ID of the most-bound star (to use as center). 
    #cen_gal_cat_s = cen_gal_cat[cen_gal_cat['gen'] == 'VELA_v2'][cen_gal_cat['galaxynumber'] == '11'][cen_gal_cat['type'] == 'Thick'][cen_gal_cat['aexp']==0.25] #takes data out of catalog
    important_col1=['disc_Lx', 'disc_Ly', 'disc_Lz']
    important_col2=['center[0](code)', 'center[1](code)', 'center[2](code)']
    rvir=cen_gal_cat_s['r_vir[kpc]'].values[0]
#cen_gal_cat_s[important_col] #prints a table with the values of the info written in important_col
    L=cen_gal_cat_s[important_col1].values[0] #taking info written in important_col1 into a list
    center=cen_gal_cat_s[important_col2].values[0]#taking info written in important_col2 into a list
    center = ds.arr(center, 'code_length').in_units('kpc')#giving center_new units of kpc (from code length)
    sp = ds.sphere(center=center,radius =(1.5*rvir, 'kpc'))
    return sp
#print L
#print center

### The catalog's R_vir:

In [7]:
rvir=cen_gal_cat_s['r_vir[kpc]'].values[0]

NameError: name 'cen_gal_cat_s' is not defined

## DM functions

In [12]:
# =====================================================================================>
   #The 2 radii to calculate, all you need is here.
# =====================================================================================>

# Only thing to change: min_bin_dis = for DM you can take 100 pc? and find_DM_1min_r_by_der(per_thresh = should be same as rvir. its (dln\rho)/(dln r) == per_thresh boundry)
# SP should be a sphere with 1.5 - 2 of the expected Rvir...
# in case of emergency: sv_w, sv_p = 41, 4 in find_DM_1min_r_by_der which talk about smoothing window for savgol, and the polonomyal degree (4 , try to change the other, this works)

#find_DM_1min_r_by_der(sp, per_thresh=1.01, min_bin_dis=0.025, xk=None, yk=None):
#find_DM_r_by_r_change_search(sp, per_thresh=0.1, min_bin_dis=0.025, xk=None, yk=None):
# =====================================================================================>


def get_mass_binned_DM(sp, min_bin_dis = 0.025):
    sp_r = sp.radius.in_units('kpc').v
    stars_r = sp[('darkmatter', 'particle_position_spherical_radius')].in_units('kpc').v
    stars_mass = sp[('darkmatter', 'particle_mass')].in_units('Msun').v
#     #cumsum
#     stars_r_argsorted = np.argsort(stars_r)
#     stars_mass_sorted = stars_mass[stars_r_argsorted]
#     stars_r_sorted    = stars_r[stars_r_argsorted]
#     stars_mass_cumsum = np.cumsum(stars_mass_sorted)
    # Binning (min_bin_dis pc bin)
    hist_bins = int(sp_r/min_bin_dis)
    hist_range = (0, sp_r) #was 100
    star_mass_binned, star_r_bin_edges = np.histogram(stars_r, bins=hist_bins, \
                                                    normed=False, range=hist_range, weights=stars_mass)
    stars_mass_binned_cumsum = np.cumsum(star_mass_binned)
    star_r_binning = 0.5*(star_r_bin_edges[1:]+star_r_bin_edges[:-1])
    yk = stars_mass_binned_cumsum
    xk = star_r_binning

#   star_volume = (4*np.pi/3) * (star_r_bin_edges[1:]**3 - star_r_bin_edges[:-1]**3)
#   star_density = np.divide(star_mass_binned, star_volume)
    return (xk,yk)



# Mass radius 0:
def find_DM_r_by_r_change_search(sp, per_thresh=0.1, min_bin_dis=0.025 , xk=None, yk=None): #min_bin_dis=0.025
    R_MIN_RES = min_bin_dis*8 # 0.2 # kpc = 200 pc

    sp_r = sp.radius.in_units('kpc').v
    stars_r = sp[('darkmatter', 'particle_position_spherical_radius')].in_units('kpc').v
    stars_mass = sp[('darkmatter', 'particle_mass')].in_units('Msun').v
    stars_r_argsorted = np.argsort(stars_r)
    stars_mass_sorted = stars_mass[stars_r_argsorted]
    stars_r_sorted    = stars_r[stars_r_argsorted]
    stars_mass_cumsum = np.cumsum(stars_mass_sorted)

    r_cur = 1.5*R_MIN_RES
    DR_MIN_RES = R_MIN_RES/8
    MAX_ITER = 1000
    #find the total_mass point to r_searched
    r_width = max(0.1*r_cur, R_MIN_RES)
    #print r_cur-.5*r_width
    m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted, r_cur-.5*r_width)]
    m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted, r_cur+.5*r_width)-1]
    slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur+.5*r_width) - np.log10(r_cur-.5*r_width))
    #slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )
    #print 'slope', slope
    while (per_thresh < slope and r_cur < 0.5*sp_r):
        #print '1.1*r_cur', r_cur
        #print 'slope', slope
        r_width = max(0.1*r_cur, R_MIN_RES) #TODO #TODO
        r_cur_1 = max(0.00001,    r_cur-.5*r_width)
        r_cur_2 = min(max(sp_r-1,0), r_cur+.5*r_width)
        m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_1)]
        m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_2)]
        slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur_2) - np.log10(r_cur_1))
        r_cur = 1.1*r_cur

    if  sp_r <= r_cur:
        slope = 0
        per_thresh = 0

        print 'BLAT', slope, r_cur, sp_r  # TODO RASIE EXCEPTION OR SOMTHING

    dr = .5*r_cur
    #r_cur = r_cur-dr
    #dr = .5*r_cur
    i=0
    epsi = 0.01*per_thresh
    while (DR_MIN_RES < dr and i < MAX_ITER and r_cur < sp_r):
        if (slope+epsi < per_thresh):
            #print 'slope < per_thresh', r_cur, slope, per_thresh
            r_cur = r_cur-dr
        elif (slope-epsi > per_thresh):
            #print 'per_thresh <= slope', r_cur, slope, per_thresh
            r_cur= r_cur+dr
        else:
            # THATS FINE, CHECK IF IT DOES HAPPEN AT ALL
            #print 'bla bla bla' # TODO RASIE EXCEPTION OR SOMTHING
            dr = 0
        r_cur_2 = min(max(sp_r-1,0), r_cur+.5*r_width)

        r_width = max(0.1*r_cur, R_MIN_RES)
        r_cur_1 = max(0.00001,    r_cur-.5*r_width)
        m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_1)]
        m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_2)]
        slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur_2) - np.log10(r_cur_1))
        #slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )
        #print 'slope', slope
        dr = 0.5*dr
        i += 1

    if (MAX_ITER == i):
        print 'MAX_ITER == i', MAX_ITER, i

    return r_cur



# Rho_1st_min 1:
from scipy.signal import savgol_filter

def find_DM_1min_r_by_der(sp, per_thresh=1.01, min_bin_dis=0.025, xk=None, yk=None):
    sv_w, sv_p = 41, 4 #31 4
    #print 'find_1min_r_by_der'

    if (xk is None or yk is None):
        xk, yk = get_mass_binned_DM(sp, min_bin_dis)

    sg_fit_d1 = savgol_filter(yk, window_length=sv_w, polyorder=sv_p, \
                            deriv=1, mode='nearest', delta=min_bin_dis) # *10.0**-3
    sg_fit_log_rho_d2 = savgol_filter(np.log10((sg_fit_d1)/xk**2), window_length=sv_w, polyorder=3, \
                            deriv=1, mode='nearest', delta=min_bin_dis) # *10.0**-3
    #counter = 1
    for i in range(len(sg_fit_log_rho_d2)-1):
        # if the secound derivitive is closed enough to the 0
        if (abs(sg_fit_log_rho_d2[i]) < (per_thresh-1)):
            #counter = counter + 1
            return .5*(xk[i]+xk[i+1])

        # if the sign is changeing
        if ((sg_fit_log_rho_d2[i]) * (sg_fit_log_rho_d2[i+1]) < 0):
            #counter = counter + 1
            return .5*(xk[i]+xk[i+1])

    return np.inf

## MASS functions

In [16]:
def find_MASS_r_by_r_change_search(sp, per_thresh=0.1, min_bin_dis=0.025 , xk=None, yk=None): #min_bin_dis=0.025
    R_MIN_RES = min_bin_dis*8 # 0.2 # kpc = 200 pc

    sp_r = sp.radius.in_units('kpc').v
    stars_r = sp[('all', 'particle_position_spherical_radius')].in_units('kpc').v
    stars_mass = sp[('all', 'particle_mass')].in_units('Msun').v
    stars_r_argsorted = np.argsort(stars_r)
    stars_mass_sorted = stars_mass[stars_r_argsorted]
    stars_r_sorted    = stars_r[stars_r_argsorted]
    stars_mass_cumsum = np.cumsum(stars_mass_sorted)

    r_cur = 1.5*R_MIN_RES
    DR_MIN_RES = R_MIN_RES/8
    MAX_ITER = 1000
    #find the total_mass point to r_searched
    r_width = max(0.1*r_cur, R_MIN_RES)
    #print r_cur-.5*r_width
    m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted, r_cur-.5*r_width)]
    m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted, r_cur+.5*r_width)-1]
    slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur+.5*r_width) - np.log10(r_cur-.5*r_width))
    #slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )
    print 'slope', slope
    while (per_thresh < slope and r_cur < 0.5*sp_r):
        #print '1.1*r_cur', r_cur
        #print 'slope', slope
        r_width = max(0.1*r_cur, R_MIN_RES) #TODO #TODO
        r_cur_1 = max(0.00001,    r_cur-.5*r_width)
        r_cur_2 = min(max(sp_r-1,0), r_cur+.5*r_width)
        m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_1)]
        m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_2)]
        slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur_2) - np.log10(r_cur_1))
        r_cur = 1.1*r_cur

    if  sp_r <= r_cur:
        slope = 0
        per_thresh = 0

        print 'BLAT', slope, r_cur, sp_r  # TODO RASIE EXCEPTION OR SOMTHING

    dr = .5*r_cur
    #r_cur = r_cur-dr
    #dr = .5*r_cur
    i=0
    epsi = 0.01*per_thresh
    while (DR_MIN_RES < dr and i < MAX_ITER and r_cur < sp_r):
        if (slope+epsi < per_thresh):
            #print 'slope < per_thresh', r_cur, slope, per_thresh
            r_cur = r_cur-dr
        elif (slope-epsi > per_thresh):
            #print 'per_thresh <= slope', r_cur, slope, per_thresh
            r_cur= r_cur+dr
        else:
            # THATS FINE, CHECK IF IT DOES HAPPEN AT ALL
            #print 'bla bla bla' # TODO RASIE EXCEPTION OR SOMTHING
            dr = 0
        r_cur_2 = min(max(sp_r-1,0), r_cur+.5*r_width)

        r_width = max(0.1*r_cur, R_MIN_RES)
        r_cur_1 = max(0.00001,    r_cur-.5*r_width)
        m_cur_1 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_1)]
        m_cur_2 = stars_mass_cumsum[np.searchsorted(stars_r_sorted,  r_cur_2)]
        slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )/ (np.log10(r_cur_2) - np.log10(r_cur_1))
        #slope = ( np.log10(m_cur_2)-np.log10(m_cur_1) )
        #print 'slope', slope
        dr = 0.5*dr
        i += 1

    if (MAX_ITER == i):
        print 'MAX_ITER == i', MAX_ITER, i

    return r_cur



# Rho_1st_min 1:
from scipy.signal import savgol_filter
